In [1]:
import gc
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy.random import seed

from tensorflow import set_random_seed

from keras.models import Sequential
from keras.layers import Dense, CuDNNGRU
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

from tqdm import tqdm

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
seed(639)
set_random_seed(5944)

In [4]:

print(os.listdir("../input"))

['unarch', 'lanl-features.zip', 'test.zip', 'sample_submission.csv', 'train.csv.zip', 'test', 'train']


In [5]:
%%time
float_data = pd.read_csv("../input/train/train.csv", dtype={"acoustic_data": np.float32, "time_to_failure": np.float32}).values

CPU times: user 2min 21s, sys: 9.22 s, total: 2min 30s
Wall time: 2min 30s


In [6]:
def extract_features(z):
    print()
    print("extract_features, z.shape:", z.shape)
    #print()
    f1 = z.mean(axis=1)
    f2 = np.transpose(np.percentile(np.abs(z), q=[0, 50, 75, 100], axis=1))
    f3 = z.std(axis=1)
    print("extract_features, f1.shape:", f1.shape)
    print("extract_features, f1:", f1)
    print("extract_features, f2.shape:", f2.shape)
    print("extract_features, f3:", f3)
    print("extract_features, f2.shape:", f2.shape)
    print("extract_features, f3:", f3)
    return np.c_[
        f1,
        f2,
        f3
    ]

In [7]:
def create_X(x, last_index=None, n_steps=150, step_length=1000):
    if last_index == None:
        last_index=len(x)
    
    print("\n")
    print("createX, x.shape:", x.shape)
    print("createX, last_index:", last_index)
    print("createX, n_steps:", n_steps)
    print("createX, step_length:", step_length)
    assert last_index - n_steps * step_length >= 0

    # Reshaping and approximate standardization with mean 5 and std 3.
    # ORIGINAL: I changed this becuase I got an No OpKernel was registered to support Op 'CudnnRNN' error
    #temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1) - 5 ) / 3
    # MY CHANGE: This doesn't fix things, I get the same errors
    #temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1).astype(np.float32) - 5 ) / 3
    temp = x[(last_index - n_steps * step_length):last_index]
    print("createX, temp.shape before reshape:", temp.shape)
    temp = temp.reshape(n_steps, -1).astype(np.float32)
    print("create_X, temp.shape after reshape:", temp.shape)
    temp = (temp - 5) / 3
    
    # Extracts features of sequences of full length 1000, of the last 100 values and finally also 
    # of the last 10 observations.
    print("createX, extract_features(temp).shape:", extract_features(temp).shape)
    print("createX, extract_features(temp[:, -step_length // 10:]).shape", extract_features(temp[:, -step_length // 10:]).shape)
    print("createX, extract_features(temp[:, -step_length // 100:]).shape", extract_features(temp[:, -step_length // 100:]).shape)
    print()
    result = np.c_[extract_features(temp),
                 extract_features(temp[:, -step_length // 10:]),
                 extract_features(temp[:, -step_length // 100:])]
    print("createX, result shape:", result.shape)
    print()
    '''
    return np.c_[extract_features(temp),
                 extract_features(temp[:, -step_length // 10:]),
                 extract_features(temp[:, -step_length // 100:])]
    '''
    return result

In [8]:
# Query "create_X" to figure out the number of features
n_features = create_X(float_data[0:150000]).shape[1]
print("Our RNN is based on %i features"% n_features)
    
# The generator endlessly selects "batch_size" ending positions of sub-time series. For each ending position,
# the "time_to_failure" serves as target, while the features are created by the function "create_X".
def generator(data, min_index=0, max_index=None, batch_size=16, n_steps=150, step_length=1000):
    if max_index is None:
        max_index = len(data) - 1
    print("\n")   
    print("generator, data.shape:", data.shape)
    while True:
        # Pick indices of ending positions
        rows = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)
        print("generator, type(rows):", type(rows))
        print("generator, rows.shape:", rows.shape)
        print("generator rows:\n", rows)
         
        # Initialize feature matrices and targets
        samples = np.zeros((batch_size, n_steps, n_features))
        targets = np.zeros(batch_size, )
        print("generator, samples.shape:", samples.shape)
        print("generator, targets.shape:", targets.shape)
        
        for j, row in enumerate(rows):
            print("generator j: {}, row: {}".format(j, row))
            samples[j] = create_X(data[:, 0], last_index=row, n_steps=n_steps, step_length=step_length)
            print("generator, samples[{}].shape:".format(j), samples[j].shape)
            print("generator, row - 1:", row - 1)
            targets[j] = data[row - 1, 1]
            print("generator, targets[{}].shape:".format(j), targets[j].shape)
            print()
        yield samples, targets
        
batch_size = 32

# Position of second (of 16) earthquake. Used to have a clean split
# between train and validation
second_earthquake = 50085877
float_data[second_earthquake, 1]

# Initialize generators
# train_gen = generator(float_data, batch_size=batch_size) # Use this for better score
train_gen = generator(float_data, batch_size=batch_size, min_index=second_earthquake + 1)
valid_gen = generator(float_data, batch_size=batch_size, max_index=second_earthquake)



createX, x.shape: (150000, 2)
createX, last_index: 150000
createX, n_steps: 150
createX, step_length: 1000
createX, temp.shape before reshape: (150000, 2)
create_X, temp.shape after reshape: (150, 2000)

extract_features, z.shape: (150, 2000)
extract_features, f1.shape: (150,)
extract_features, f1: [-0.5656501  -0.53015035 -0.5859838  -0.52781725 -0.6011343  -0.5656503
 -0.5451505  -0.530984   -0.5649652  -0.5653336  -0.6020004  -0.61516726
 -0.6277976  -0.61201686 -0.58185035 -0.5916839  -0.5591199  -0.6201836
 -0.5798504  -0.63835055 -0.6369459  -0.63486683 -0.5745337  -0.58403385
 -0.5477712  -0.6117002  -0.57703376 -0.6098672  -0.63842744 -0.58671683
 -0.56671697 -0.5708838  -0.5909266  -0.58806676 -0.60606694 -0.6142339
 -0.62459016 -0.6220668  -0.60390025 -0.5757338  -0.6359081  -0.60658336
 -0.60841686 -0.63158387 -0.57541734 -0.63009    -0.64593357 -0.6357672
 -0.611434   -0.6324083  -0.63826686 -0.62660044 -0.634434   -0.6090717
 -0.6126169  -0.6364504  -0.6704507  -0.673737

In [ ]:
cb = [ModelCheckpoint("model.hdf5", save_best_only=True, period=3)]

model = Sequential()
model.add(CuDNNGRU(48, input_shape=(None, n_features)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.summary()

In [ ]:
model.compile(optimizer=adam(lr=0.0005), loss="mae")

history = model.fit_generator(train_gen,
                              steps_per_epoch=1000,
                              epochs=30,
                              verbose=0,
                              callbacks=cb,
                              validation_data=valid_gen,
                              validation_steps=200)

In [ ]:
def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None

perf_plot(history)

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

In [ ]:
for i, seg_id in enumerate(tqdm(submission.index)):
  #  print(i)
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    x = seg['acoustic_data'].values
    submission.time_to_failure[i] = model.predict(np.expand_dims(create_X(x), 0))

submission.head()

In [ ]:
submission.to_csv('submission.csv')

In [ ]:
pd.read_csv?

In [ ]:
train_gen.__next__()

In [ ]:
gc.collect()